In [74]:
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
import random

In [24]:
df = pd.read_csv("marketing_campaign.csv", sep=";")

In [3]:
df

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,2013-06-13,46,709,...,5,0,0,0,0,0,0,3,11,0
2236,4001,1946,PhD,Together,64014.0,2,1,2014-06-10,56,406,...,7,0,0,0,1,0,0,3,11,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,2014-01-25,91,908,...,6,0,1,0,0,0,0,3,11,0
2238,8235,1956,Master,Together,69245.0,0,1,2014-01-24,8,428,...,3,0,0,0,0,0,0,3,11,0


In [85]:
def prep_data(df):
    #transform columns
    df.columns = df.columns.str.lower()
    df["dt_customer"] = pd.to_datetime(df["dt_customer"]).dt.year

    #filling nan's in income
    df["income"] = df["income"].fillna(df["income"].median())

    #data reliability check
    df = df.drop(columns=["id", "z_costcontact", "z_revenue"])
    df = df[df["year_birth"]>1930]
    df = df[df["income"]<200000]
    martial_ac = ["Single","Together","Married","Divorced","Widow"]
    df = df[df["marital_status"].isin(martial_ac)]

    X = df.drop(columns="response", axis=1)

    numeric_features = X.select_dtypes(include=['number']).columns.tolist()
    categorical_features = X.select_dtypes(include=['object']).columns.tolist()

    #one hot encoding
    encoder = OneHotEncoder(sparse_output=False,drop='first',handle_unknown='ignore')
    encoded_features = encoder.fit_transform(df[categorical_features])
    encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features),index=df.index)

    #df.reset_index(drop=True, inplace=True)
    df.drop(columns=categorical_features, inplace=True)
    df = pd.concat([df, encoded_df], axis=1)

    #data split
    y = df["response"]
    X = df.drop(columns="response", axis=1)

    x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=2115)

    #scaling
    scaler = StandardScaler()
    x_train[numeric_features] = scaler.fit_transform(x_train[numeric_features])
    x_test[numeric_features] = scaler.transform(x_test[numeric_features])

    return x_train, y_train, x_test, y_test    

In [71]:
df = pd.read_csv("marketing_campaign.csv", sep=";")

In [90]:
x_train, y_train, x_test, y_test = prep_data(df)

In [91]:
x_train

,year_birth,income,kidhome,teenhome,dt_customer,recency,mntwines,mntfruits,mntmeatproducts,mntfishproducts,...,acceptedcmp2,complain,education_Basic,education_Graduation,education_Master,education_PhD,marital_status_Married,marital_status_Single,marital_status_Together,marital_status_Widow
194,0.359956,-0.034330,1.020172,0.922158,0.0,-0.392621,-0.196788,-0.515645,-0.536848,-0.539086,...,-0.119251,-0.078789,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
785,1.473898,-1.473871,1.020172,-0.933606,0.0,-1.533264,-0.893193,-0.368589,-0.712671,-0.631299,...,-0.119251,-0.078789,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1388,0.959771,-0.324084,1.020172,0.922158,0.0,-0.496316,-0.847972,-0.638192,-0.659924,-0.649742,...,-0.119251,-0.078789,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
571,-0.068483,-0.629629,1.020172,0.922158,0.0,0.298677,-0.808780,-0.662701,-0.712671,-0.686627,...,-0.119251,-0.078789,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1662,0.617019,0.609652,-0.826494,0.922158,0.0,0.955411,0.683516,-0.074477,0.276335,-0.096464,...,-0.119251,-0.078789,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1647,1.045459,0.318773,-0.826494,-0.933606,0.0,0.748021,0.538808,0.195126,0.052160,-0.170234,...,-0.119251,-0.078789,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
198,-1.268112,1.632320,-0.826494,-0.933606,0.0,-1.014790,0.131818,3.969569,1.384022,0.235502,...,-0.119251,-0.078789,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
763,0.788395,-0.949043,1.020172,-0.933606,0.0,-1.291309,-0.787677,-0.638192,-0.695089,-0.649742,...,-0.119251,-0.078789,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
885,0.102893,1.414629,-0.826494,-0.933606,0.0,0.609762,1.774852,0.685314,2.276326,2.411727,...,-0.119251,-0.078789,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


2239

## make_pipeline test

In [ ]:
class ColumnRemover(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_remove):
        self.columns_to_remove = columns_to_remove

    def fit(self, X, y=None):
        return self

    def prep_data(X):
        pass

    def transform(self, X):
        return X.drop(self.columns_to_remove, axis=1)

In [65]:
class PrintData(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        #print(X)
        return X

In [43]:
preprocess = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(categories=[categories.get(col, 'auto') for col in categorical_features]), categorical_features))

In [67]:
pipe = make_pipeline(
    preprocess,
    PrintData(),
    verbose=False
)

In [68]:
pipe.fit(df)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['year_birth', 'income',
                                                   'kidhome', 'teenhome',
                                                   'dt_customer', 'recency',
                                                   'mntwines', 'mntfruits',
                                                   'mntmeatproducts',
                                                   'mntfishproducts',
                                                   'mntsweetproducts',
                                                   'mntgoldprods',
                                                   'numdealspurchases',
                                                   'numwebpurchases',
                                                   'numcatalogpurchases',
                                                   'numstorepurchases',
                                                   'numw...
                                                   'acceptedcmp1',
                                                   'acceptedcmp2', 'complain',
                                                   'z_costcontact', 'z_revenue',
                                                   'response']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Graduation', 'PhD', 'Master', 'Basic', '2n Cycle'], dtype=object),
                                                                            array(['Single', 'Together', 'Married', 'Divorced', 'Widow', 'Alone',
       'Absurd', 'YOLO'], dtype=object)]),
                                                  ['education',
                                                   'marital_status'])])),
                ('printdata', PrintData())])

In [70]:
xd = pipe.fit_transform(df)

In [72]:
pd.DataFrame(xd)

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,-0.985345,0.234063,-0.825218,-0.929894,-1.502225,0.307039,0.983781,1.551577,1.679702,2.462147,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.235733,-0.234559,1.032559,0.906934,1.420036,-0.383664,-0.870479,-0.636301,-0.713225,-0.650449,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.317643,0.769478,-0.825218,-0.929894,-0.041094,-0.798086,0.362723,0.570804,-0.177032,1.345274,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.268149,-1.017239,1.032559,-0.929894,1.420036,-0.798086,-0.870479,-0.560857,-0.651187,-0.503974,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.017761,0.240221,1.032559,-0.929894,1.420036,1.550305,-0.389085,0.419916,-0.216914,0.155164,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,-0.150717,0.356642,-0.825218,0.906934,-0.041094,-0.107383,1.203678,0.419916,0.066692,0.081926,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2236,-1.903435,0.467539,2.890335,0.906934,1.420036,0.237969,0.303291,-0.661449,-0.606873,-0.687068,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2237,1.017761,0.188091,-0.825218,-0.929894,1.420036,1.446700,1.795020,0.545656,0.221789,-0.101168,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2238,-1.068807,0.675388,-0.825218,0.906934,1.420036,-1.419719,0.368666,0.092992,0.208495,0.777683,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
